# HW06: Transformers and Doc Embeddings

Remember that these homework work as a completion grade. **You can skip one section of this homework.**

In [1]:
!wget https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv

import pandas as pd
import nltk
df = pd.read_csv('train.csv')

df.columns = ["label", "title", "lead"]
label_map = {1:"world", 2:"sport", 3:"business", 4:"sci/tech"}
def replace_label(x):
	return label_map[x]
df["label"] = df["label"].apply(replace_label) 
df["text"] = df["title"] + " " + df["lead"]
df = df.sample(n=10000) # # only use 10K datapoints
df.head()

--2023-03-28 09:58:53--  https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29470338 (28M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]  28.10M  --.-KB/s    in 0.1s    

2023-03-28 09:58:54 (207 MB/s) - ‘train.csv’ saved [29470338/29470338]



,label,title,lead,text
17598,sport,Lions run over Bills,(09/03/04) The Lions closed out their exhibiti...,Lions run over Bills (09/03/04) The Lions clos...
27956,sci/tech,"IBM, LG Electronics Put Brakes On Joint Venture",IBM Corp. and LG Electronics Inc. will end the...,"IBM, LG Electronics Put Brakes On Joint Ventur..."
83999,business,Microsoft seeks pre-dividend shift in stock plan,"SEATTLE - For Microsoft Corp. shareholders, th...",Microsoft seeks pre-dividend shift in stock pl...
57855,world,Slain Policemen Laid to Rest in Haiti (AP),AP - Gunfire crackled around Haiti's capital a...,Slain Policemen Laid to Rest in Haiti (AP) AP ...
81322,sport,Slumping Bayern face the hottest Bundesliga team,Felix Magath wanted a victory against Juventus...,Slumping Bayern face the hottest Bundesliga te...


## Hugginface Transformers

In [4]:
#!pip install transformers
from transformers import DistilBertForSequenceClassification, DistilBertConfig, DistilBertTokenizerFast
import torch

In [5]:
##TODO build a transformer model to do sequence classification with the goal to predict the label from the text
config = DistilBertConfig.from_pretrained('distilbert-base-uncased')
config.num_labels = 4
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', config=config)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifi

In [12]:
##TODO print the summary of the model
print(model)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [6]:
##TODO split the sample into a training and a test set 
from sklearn.model_selection import train_test_split

label2id = {"world":0, "sport":1, "business":2, "sci/tech":3}
df['label_id'] = df['label'].apply(lambda x: label2id[x])

X_train, X_test, y_train, y_test = train_test_split(df['text'].tolist(), df['label_id'].tolist(), test_size=.2)


##TODO prepare the dataset.
class BertDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(X_train, padding=True, truncation=True)
test_encodings = tokenizer(X_test, padding=True, truncation=True)
train_dataset = BertDataset(train_encodings, y_train)
test_dataset = BertDataset(test_encodings, y_test)

In [8]:
##TODO fit the model and print the obtained accuracy (hint: you can follow the training steps in the notebook. To learn more, checkout the trainer class of huggingface transformers)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print (device)

# trainer automatically use cuda
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
  output_dir='./results',
  num_train_epochs=1,              # total number of training epochs
  per_device_train_batch_size=16,  # batch size per device during training
  per_device_eval_batch_size=64,   # batch size for evaluation
  warmup_steps=500,                # number of warmup steps for learning rate scheduler
  weight_decay=0.01,               # strength of weight decay
  logging_dir='.',            # directory for storing logs
  logging_steps=150,
  evaluation_strategy='no',
)

trainer = Trainer(
  model=model,                         # the instantiated (hug) Transformers model to be trained
  args=training_args,                  # training arguments, defined above
  train_dataset=train_dataset,         # training dataset
)

trainer.train()

cuda


Step,Training Loss
150,0.277200
300,0.213300
450,0.248900


TrainOutput(global_step=500, training_loss=0.254818603515625, metrics={'train_runtime': 242.2205, 'train_samples_per_second': 33.028, 'train_steps_per_second': 2.064, 'total_flos': 695478647808000.0, 'train_loss': 0.254818603515625, 'epoch': 1.0})

In [22]:
from sklearn.metrics import accuracy_score
from transformers import TextClassificationPipeline
from tqdm import tqdm

preds = []
model.to('cpu')
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)
results = pipe(X_test)
for result in results:
  pred = np.argmax([dic['score'] for dic in result])
  preds.append(pred)

/usr/local/lib/python3.9/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [23]:
accuracy_score(y_test, preds)

0.9015

# Doc Embedding

In [1]:
# obtain the data
!wget http://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.eval.v1.1.zip
!wget http://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.gs.zip

!unzip sts2017.eval.v1.1.zip 
!unzip sts2017.gs.zip 

--2023-03-28 12:20:34--  http://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.eval.v1.1.zip
Resolving alt.qcri.org (alt.qcri.org)... 80.76.166.234
Connecting to alt.qcri.org (alt.qcri.org)|80.76.166.234|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.eval.v1.1.zip [following]
--2023-03-28 12:20:34--  https://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.eval.v1.1.zip
Connecting to alt.qcri.org (alt.qcri.org)|80.76.166.234|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87902 (86K) [application/zip]
Saving to: ‘sts2017.eval.v1.1.zip’

sts2017.eval.v1.1.z 100%[===================>]  85.84K   195KB/s    in 0.4s    

2023-03-28 12:20:36 (195 KB/s) - ‘sts2017.eval.v1.1.zip’ saved [87902/87902]

URL transformed to HTTPS due to an HSTS policy
--2023-03-28 12:20:36--  https://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.gs.zip
Resolving alt.qcri.org

In [2]:
# load the data

def load_STS_data():
    with open("STS2017.gs/STS.gs.track5.en-en.txt") as f:
        labels = [float(line.strip()) for line in f]
    
    text_a, text_b = [], []
    with open("STS2017.eval.v1.1/STS.input.track5.en-en.txt") as f:
        for line in f:
            line = line.strip().split("\t")
            text_a.append(line[0])
            text_b.append(line[1])
    return text_a, text_b, labels

text_a, text_b, labels = load_STS_data()
text_a[0], text_b[0], labels[0]

('A person is on a baseball team.',
 'A person is playing basketball on a team.',
 2.4)

In [3]:
# some utils
from scipy.stats import spearmanr
def evaluate(predictions, labels):
    print ("spearman's rank correlation", spearmanr(predictions, labels)[0])

import numpy as np
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(a,b):
    return dot(a, b)/(norm(a)*norm(b))


In [4]:
# Wordcounts baseline
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
vec.fit(text_a + text_b)

# encode documents
text_a_encoded = np.array(vec.transform(text_a).todense())
text_b_encoded = np.array(vec.transform(text_b).todense())

# predict cosine similarities
predictions = [cosine_similarity(a,b) for a,b in zip(text_a_encoded, text_b_encoded)]

# evaluate
evaluate(predictions, labels)

spearman's rank correlation 0.6998056665685976


In [7]:
import nltk
nltk.download('punkt')
##TODO train Doc2Vec on the texts in the dataset
##TODO derive the word vectors for each text in the dataset
##TODO compute cosine similarity between the text pairs and evaluate spearman's rank correlation
## Don't worry if results are not satisfactory using Doc2Vec (the dataset is too small to train good embeddings)
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk import word_tokenize
docs = []
for i in text_a + text_b:
    docs.append(word_tokenize(i))

doc_iterator = [TaggedDocument(doc, [i]) for i, doc in enumerate(docs)]
d2v = Doc2Vec(doc_iterator,
                min_count=2, # minimum word count
                window=5,    # window size
                vector_size=25, # size of document vector
                sample=1e-4, 
                negative=5, 
                workers=4, # threads
                #dbow_words = 1 # uncomment to get word vectors too
                max_vocab_size=1000) # max vocab size

text_a_encoded = d2v.dv.vectors[:len(text_a)]
text_b_encoded = d2v.dv.vectors[len(text_a):]

# predict cosine similarities
predictions = [cosine_similarity(a,b) for a,b in zip(text_a_encoded, text_b_encoded)]

# evaluate
evaluate(predictions, labels)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


spearman's rank correlation 0.014730844630254609


In [8]:
##TODO do the same with embeddings provided by spaCy
import spacy
nlp = spacy.load('en_core_web_sm')

text_a_encoded = [nlp(text).vector for text in text_a]
text_b_encoded = [nlp(text).vector for text in text_b]

In [9]:
# predict cosine similarities
predictions = [cosine_similarity(a,b) for a,b in zip(text_a_encoded, text_b_encoded)]

# evaluate
evaluate(predictions, labels)

spearman's rank correlation 0.48796451382976114


In [13]:
##TODO do the same with SBERT embeddings
# import locale
# def getpreferredencoding(do_setlocale = True):
#     return "UTF-8"
# locale.getpreferredencoding = getpreferredencoding
# !pip install sentence_transformers
from sentence_transformers import SentenceTransformer
model = "bert-base-nli-mean-tokens"
embedder = SentenceTransformer(model)
text_a_encoded = embedder.encode(text_a)
text_b_encoded = embedder.encode(text_b)

# predict cosine similarities
predictions = [cosine_similarity(a,b) for a,b in zip(text_a_encoded, text_b_encoded)]

# evaluate
evaluate(predictions, labels)

spearman's rank correlation 0.8008164100246977
